INSTITUTO TECNOLOGICO Y DE ESTUDIOS SUPERIORES DE MONTERREY

 CAMPUS PUEBLA

ANALITICA DE DATOS Y HERRAMIENTAS DE INTELIGENCIA ARTIFICIAL II

CATEGORIZACIÓN DE COLUMNAS NUMÉRICAS POR MÉTODO DE STURGES

ALUMNO:

Ángel Roberto González Angulo | A01735880

PROFESOR:

Alfredo García Suárez

GRUPO:
501


FECHA DE ENTREGA:
aa/aa/aaaa

# Librerías y lectura de datos

In [255]:
# Importar las librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [256]:
df = pd.read_excel("OttawaNoAtipicos.xlsx")
df.drop("Unnamed: 0", axis = 1, inplace = True)
df.head(6)

,last_scraped,source,host_name,host_response_time,host_verifications,neighbourhood_cleansed,property_type,room_type,bathrooms_text,amenities,...,beds,amenities_num,room_type_num,neighbourhood_cleansed_num,host_response_time_num,bathroom_text_clean,bathroom_text_num,neighbourhood_Category,neighbourhood_Category_num,priceMex
0,2024-06-21,city scrape,Glenn,within an hour,"['email', 'phone']",Kitchissippi,Entire rental unit,Entire home/apt,1 bath,"[""Kitchen"", ""Shampoo"", ""Coffee maker"", ""Smoke ...",...,3,28,0,12,3,bath,3,Oeste,2,2549.40
1,2024-06-21,city scrape,Daniel,within a day,"['email', 'phone']",Rideau-Goulbourn,Entire home,Entire home/apt,1 bath,"[""Extra pillows and blankets"", ""Kitchen"", ""Sha...",...,2,61,0,16,1,bath,3,Sur,3,2913.60
2,2024-06-21,city scrape,Ela,within an hour,"['email', 'phone']",Capital,Private room in home,Private room,1 private bath,"[""Free washer \u2013 In building"", ""Extra pill...",...,1,49,1,4,3,private bath,4,Oeste,2,1078.03
3,2024-06-21,city scrape,William,within an hour,['phone'],Gloucester-South Nepean,Entire home,Entire home/apt,1 bath,"[""Kitchen"", ""Shampoo"", ""Host greets you"", ""Smo...",...,4,27,0,7,3,bath,3,Oeste,2,961.49
4,2024-06-21,city scrape,Greg And Valerie,within an hour,"['email', 'phone', 'work_email']",Kitchissippi,Entire home,Entire home/apt,1 bath,"[""Extra pillows and blankets"", ""Self check-in""...",...,2,45,0,12,3,bath,3,Oeste,2,1325.69
5,2024-06-21,city scrape,Greg And Valerie,within an hour,"['email', 'phone', 'work_email']",Kitchissippi,Entire townhouse,Entire home/apt,1 bath,"[""Self check-in"", ""Free driveway parking on pr...",...,2,57,0,12,3,bath,3,Oeste,2,1879.27


In [257]:
# Revisamos el nomnbre de todas las columnas
df.columns

Index(['last_scraped', 'source', 'host_name', 'host_response_time',
       'host_verifications', 'neighbourhood_cleansed', 'property_type',
       'room_type', 'bathrooms_text', 'amenities', 'calendar_last_scraped',
       'host_id', 'host_since', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_listings_count', 'host_has_profile_pic',
       'host_identity_verified', 'latitude', 'longitude', 'bathrooms',
       'bedrooms', 'availability_30', 'number_of_reviews_ltm',
       'number_of_reviews_l30d', 'review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value', 'instant_bookable',
       'calculated_host_listings_count', 'host_years',
       'host_verifications_num', 'host_total_listings_count', 'accommodates',
       'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'availability_365',
       'number_of_reviews',

In [258]:
# Hacemos un dataframe que contenga aquellas columnas que serán catégoricas y que nos interesa estudiar en el analisis

dfSturges = df[["amenities_num","host_total_listings_count","accommodates","minimum_nights_avg_ntm","maximum_nights_avg_ntm","availability_365","number_of_reviews",
                "host_years","priceMex","host_response_rate","host_acceptance_rate","host_verifications_num","host_response_time_num",
                "neighbourhood_Category_num","bathroom_text_num","beds","review_scores_value","reviews_per_month","review_scores_location","review_scores_rating","review_scores_cleanliness",
                "review_scores_communication"]]
dfSturges

,amenities_num,host_total_listings_count,accommodates,minimum_nights_avg_ntm,maximum_nights_avg_ntm,availability_365,number_of_reviews,host_years,priceMex,host_response_rate,...,host_response_time_num,neighbourhood_Category_num,bathroom_text_num,beds,review_scores_value,reviews_per_month,review_scores_location,review_scores_rating,review_scores_cleanliness,review_scores_communication
0,28,1,4,3.0,180.0,99,99,13,2549.40,100.0,...,3,2,3,3,5,0.65,5,5,5,5
1,61,1,4,2.0,7.0,265,6,13,2913.60,100.0,...,1,3,3,2,5,0.04,5,5,5,5
2,49,5,1,30.0,365.0,0,9,12,1078.03,100.0,...,3,2,4,1,5,0.12,5,5,5,5
3,27,8,8,1.0,90.0,344,76,12,961.49,100.0,...,3,2,3,4,5,0.57,5,5,5,5
4,45,8,4,31.0,1125.0,175,9,12,1325.69,100.0,...,3,2,3,2,5,0.09,5,5,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2857,35,3,2,1.0,14.0,85,0,8,874.08,100.0,...,3,2,4,1,5,1.16,5,5,5,5
2858,9,3,1,30.0,70.0,69,0,4,509.88,100.0,...,3,3,3,1,5,1.16,5,5,5,5
2859,57,8,8,5.3,1125.0,365,0,8,7473.38,100.0,...,3,2,3,4,5,1.16,5,5,5,5
2860,4,2,4,1.0,6.0,329,0,10,1165.44,94.8,...,3,2,3,1,5,1.16,5,5,5,5


In [259]:
# Hacemos otro dataframe para dropear las columnas que
# Eliminar del df cuantitativo, las columnas seleccionadas:
df.drop(["amenities_num","host_total_listings_count","accommodates","minimum_nights_avg_ntm","maximum_nights_avg_ntm","availability_365","number_of_reviews",
                "host_years","priceMex","host_response_rate","host_acceptance_rate","host_verifications_num","host_response_time_num",
                "neighbourhood_Category_num","bathroom_text_num","beds","review_scores_value","reviews_per_month","review_scores_location","review_scores_rating","review_scores_cleanliness",
                "review_scores_communication"], axis = 1, inplace = True)
df

,last_scraped,source,host_name,host_response_time,host_verifications,neighbourhood_cleansed,property_type,room_type,bathrooms_text,amenities,...,number_of_reviews_l30d,review_scores_accuracy,review_scores_checkin,instant_bookable,calculated_host_listings_count,price,room_type_num,neighbourhood_cleansed_num,bathroom_text_clean,neighbourhood_Category
0,2024-06-21,city scrape,Glenn,within an hour,"['email', 'phone']",Kitchissippi,Entire rental unit,Entire home/apt,1 bath,"[""Kitchen"", ""Shampoo"", ""Coffee maker"", ""Smoke ...",...,2,5,5,0,1,175,0,12,bath,Oeste
1,2024-06-21,city scrape,Daniel,within a day,"['email', 'phone']",Rideau-Goulbourn,Entire home,Entire home/apt,1 bath,"[""Extra pillows and blankets"", ""Kitchen"", ""Sha...",...,0,5,5,0,1,200,0,16,bath,Sur
2,2024-06-21,city scrape,Ela,within an hour,"['email', 'phone']",Capital,Private room in home,Private room,1 private bath,"[""Free washer \u2013 In building"", ""Extra pill...",...,0,5,5,0,1,74,1,4,private bath,Oeste
3,2024-06-21,city scrape,William,within an hour,['phone'],Gloucester-South Nepean,Entire home,Entire home/apt,1 bath,"[""Kitchen"", ""Shampoo"", ""Host greets you"", ""Smo...",...,5,5,5,0,1,66,0,7,bath,Oeste
4,2024-06-21,city scrape,Greg And Valerie,within an hour,"['email', 'phone', 'work_email']",Kitchissippi,Entire home,Entire home/apt,1 bath,"[""Extra pillows and blankets"", ""Self check-in""...",...,0,5,5,0,6,91,0,12,bath,Oeste
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2857,2024-06-21,city scrape,Vivek,within an hour,"['email', 'phone']",Gloucester-South Nepean,Private room in home,Private room,1 private bath,"[""Extra pillows and blankets"", ""Shampoo"", ""Hos...",...,0,5,5,0,3,60,1,7,private bath,Oeste
2858,2024-06-21,city scrape,Tatiana,within an hour,"['email', 'phone']",Rideau-Vanier,Private room in rental unit,Private room,1 bath,"[""Washer"", ""Carbon monoxide alarm"", ""Kitchen"",...",...,0,5,5,0,1,35,1,18,bath,Sur
2859,2024-06-21,city scrape,Genevieve,within an hour,"['email', 'phone', 'work_email']",Osgoode,Entire home,Entire home/apt,3 baths,"[""Extra pillows and blankets"", ""Self check-in""...",...,0,5,5,1,63,513,0,15,bath,Oeste
2860,2024-06-21,city scrape,Shannon,within an hour,"['email', 'phone', 'work_email']",Kitchissippi,Entire rental unit,Entire home/apt,1 bath,"[""Free parking on premises"", ""Washer"", ""Air co...",...,0,5,5,0,1,80,0,12,bath,Oeste


In [260]:
# Comprobar que no se tengan nulos
dfSturges.isnull().sum().sum()

0

# Categorización por medio de la regla de Sturges

In [261]:
# Usando el número de amenidades
#Calculamos el número total de la población "n"

dfSturges['amenities_num'].info()
n = len(dfSturges['amenities_num'])
n
# n de CDMX: 26,536
# n de Seattle: 6,442

# Obtener el límite superior y el límite inferior de la columna objetivo
Max = dfSturges["amenities_num"].max()
Min = dfSturges["amenities_num"].min()
Limites = [Min, Max]
Limites

# CDMX: 0 al 78
# Seattle: 0 al 86

# Calculamos el rango R
R = Max - Min
R

# Regla de Sturges: Calcular el número de Intervalos de Clase "ni"
ni = 1 + 3.32*np.log10(n)
ni = int(ni)
ni

# Calculamos el ancho del intervalo "i"
i = R/ni
i = round(i,0)
i

# Categorización de variables
intervalos = np.linspace(Min, Max, ni)
intervalos = np.round(intervalos, 0)

intervalos = [ 0,  7, 14, 22, 29, 36, 43, 50, 57, 64, 72, 79, 86]

# Creamos las categorias
categorias = [ '(0 - 7) Básico', '(8 - 14) Limitado', '(15 - 22) Estándar', '(23 - 29) Completo', '(30 - 36) Amplio', 
              '(37 - 43) Enriquecido', '(44 - 50) Superior', '(51 - 57) Extendido', '(58 - 64) Excepcional', 
               '(65 - 72) Muy completo', '(73 - 79) Premium', '(80 - 86) De lujo' ]

# Finalmente creamos las categorías en la columna numérica
dfSturges['amenitiesCat'] = pd.cut(x = dfSturges['amenities_num'], bins = intervalos, labels = categorias, include_lowest=True)
dfSturges['amenitiesCat'].value_counts()

<class 'pandas.core.series.Series'>
RangeIndex: 2862 entries, 0 to 2861
Series name: amenities_num
Non-Null Count  Dtype
--------------  -----
2862 non-null   int64
dtypes: int64(1)
memory usage: 22.5 KB


C:\Users\Angel González\AppData\Local\Temp\ipykernel_24188\2398456891.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfSturges['amenitiesCat'] = pd.cut(x = dfSturges['amenities_num'], bins = intervalos, labels = categorias, include_lowest=True)


amenitiesCat
(44 - 50) Superior        537
(37 - 43) Enriquecido     470
(30 - 36) Amplio          426
(51 - 57) Extendido       326
(23 - 29) Completo        291
(15 - 22) Estándar        263
(8 - 14) Limitado         223
(58 - 64) Excepcional     176
(65 - 72) Muy completo     80
(0 - 7) Básico             47
(73 - 79) Premium          18
(80 - 86) De lujo           5
Name: count, dtype: int64

In [262]:
# Usando el número total de lugares que el host posee
#Calculamos el número total de la población "n"

dfSturges['host_total_listings_count'].info()
n = len(dfSturges['host_total_listings_count'])
n
# n de CDMX: 26,536
# n de Seattle: 6,442

# Obtener el límite superior y el límite inferior de la columna objetivo
Max = dfSturges["host_total_listings_count"].max()
Min = dfSturges["host_total_listings_count"].min()
Limites = [Min, Max]
Limites

# Calculamos el rango R
R = Max - Min
R

# Regla de Sturges: Calcular el número de Intervalos de Clase "ni"
ni = 1 + 3.32*np.log10(n)
ni = int(ni)
ni

# Calculamos el ancho del intervalo "i"
i = R/ni
i = round(i,0)
i

# Categorización de variables
intervalos = np.linspace(Min, Max, ni)
intervalos = np.round(intervalos, 0)
intervalos

categorias = [ 
    '(1 - 14) Un listado',
    '(14 - 27) Principiante',
    '(27 - 40) En crecimiento',
    '(40 - 53) Intermedio',
    '(53 - 66) Avanzado',
    '(66 - 79) Experimentado',
    '(79 - 92) Establecido',
    '(92 - 105) Profesional',
    '(105 - 118) Operador experimentado',
    '(118 - 131) Experto',
    '(131 - 144) Experto en hospedaje'
]

# Finalmente creamos las categorías en la columna numérica
dfSturges['hostTotalListingsCat'] = pd.cut(x = dfSturges['host_total_listings_count'], bins = intervalos, labels = categorias, include_lowest=True)
dfSturges['hostTotalListingsCat'].value_counts()

<class 'pandas.core.series.Series'>
RangeIndex: 2862 entries, 0 to 2861
Series name: host_total_listings_count
Non-Null Count  Dtype
--------------  -----
2862 non-null   int64
dtypes: int64(1)
memory usage: 22.5 KB


C:\Users\Angel González\AppData\Local\Temp\ipykernel_24188\3446268958.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfSturges['hostTotalListingsCat'] = pd.cut(x = dfSturges['host_total_listings_count'], bins = intervalos, labels = categorias, include_lowest=True)


hostTotalListingsCat
(1 - 14) Un listado                   2531
(14 - 27) Principiante                 159
(27 - 40) En crecimiento                83
(40 - 53) Intermedio                    33
(131 - 144) Experto en hospedaje        24
(66 - 79) Experimentado                 22
(79 - 92) Establecido                    4
(105 - 118) Operador experimentado       4
(53 - 66) Avanzado                       2
(92 - 105) Profesional                   0
(118 - 131) Experto                      0
Name: count, dtype: int64

In [263]:
# Usando el número total de espacios que hay en los airbnb
#Calculamos el número total de la población "n"

dfSturges['accommodates'].info()
n = len(dfSturges['accommodates'])
n
# n de CDMX: 26,536
# n de Seattle: 6,442

# Obtener el límite superior y el límite inferior de la columna objetivo
Max = dfSturges["accommodates"].max()
Min = dfSturges["accommodates"].min()
Limites = [Min, Max]
Limites

# Calculamos el rango R
R = Max - Min
R

# Regla de Sturges: Calcular el número de Intervalos de Clase "ni"
ni = 1 + 3.32*np.log10(n)
ni = int(ni)
ni

# Calculamos el ancho del intervalo "i"
i = R/ni
i = round(i,0)
i

# Categorización de variables
intervalos = np.linspace(Min, Max, ni)
intervalos = np.round(intervalos, 0)

intervalos = [ 1,  2,  4,  6,  8, 10, 12]

categorias = [ 
    '(1 - 2) Pareja',
    '(2 - 4) Familiar pequeño',
    '(4 - 6) Familiar mediano',
    '(6 - 8) Familiar grande',
    '(8 - 10) Grupo',
    '(10 - 12) Grupo grande'
]

# Finalmente creamos las categorías en la columna numérica
dfSturges['accommodatesCat'] = pd.cut(x = dfSturges['accommodates'], bins = intervalos, labels = categorias, include_lowest=True)
dfSturges['accommodatesCat'].value_counts()

<class 'pandas.core.series.Series'>
RangeIndex: 2862 entries, 0 to 2861
Series name: accommodates
Non-Null Count  Dtype
--------------  -----
2862 non-null   int64
dtypes: int64(1)
memory usage: 22.5 KB


C:\Users\Angel González\AppData\Local\Temp\ipykernel_24188\1843347808.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfSturges['accommodatesCat'] = pd.cut(x = dfSturges['accommodates'], bins = intervalos, labels = categorias, include_lowest=True)


accommodatesCat
(1 - 2) Pareja              1494
(2 - 4) Familiar pequeño     688
(4 - 6) Familiar mediano     405
(6 - 8) Familiar grande      191
(8 - 10) Grupo                82
(10 - 12) Grupo grande         2
Name: count, dtype: int64

In [264]:

#Calculamos el número total de la población "n"

dfSturges['minimum_nights_avg_ntm'].info()
n = len(dfSturges['minimum_nights_avg_ntm'])
n
# n de CDMX: 26,536
# n de Seattle: 6,442

# Obtener el límite superior y el límite inferior de la columna objetivo
Max = dfSturges["minimum_nights_avg_ntm"].max()
Min = dfSturges["minimum_nights_avg_ntm"].min()
Limites = [Min, Max]
Limites

# Calculamos el rango R
R = Max - Min
R

# Regla de Sturges: Calcular el número de Intervalos de Clase "ni"
ni = 1 + 3.32*np.log10(n)
ni = int(ni)
ni

# Calculamos el ancho del intervalo "i"
i = R/ni
i = round(i,0)
i

# Categorización de variables
intervalos = np.linspace(Min, Max, ni)
intervalos = np.round(intervalos, 0)
intervalos

categorias = [ 
    '(1 - 9) Muy corto plazo',
    '(9 - 17) Corto plazo',
    '(17 - 25) Plazo intermedio',
    '(25 - 33) Plazo moderado',
    '(33 - 41) Plazo extendido',
    '(41 - 50) Largo plazo',
    '(50 - 58) Muy largo plazo',
    '(58 - 66) Prolongado',
    '(66 - 74) Extensivo',
    '(74 - 82) Larga temporada',
    '(82 - 90) Temporada completa'
]

# Finalmente creamos las categorías en la columna numérica
dfSturges['minimumNightsCategory'] = pd.cut(x = dfSturges['minimum_nights_avg_ntm'], bins = intervalos, labels = categorias, include_lowest=True)
dfSturges['minimumNightsCategory'].value_counts()

<class 'pandas.core.series.Series'>
RangeIndex: 2862 entries, 0 to 2861
Series name: minimum_nights_avg_ntm
Non-Null Count  Dtype  
--------------  -----  
2862 non-null   float64
dtypes: float64(1)
memory usage: 22.5 KB


C:\Users\Angel González\AppData\Local\Temp\ipykernel_24188\1657501024.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfSturges['minimumNightsCategory'] = pd.cut(x = dfSturges['minimum_nights_avg_ntm'], bins = intervalos, labels = categorias, include_lowest=True)


minimumNightsCategory
(1 - 9) Muy corto plazo         1571
(25 - 33) Plazo moderado        1181
(9 - 17) Corto plazo              48
(17 - 25) Plazo intermedio        18
(58 - 66) Prolongado              15
(82 - 90) Temporada completa      15
(41 - 50) Largo plazo              8
(33 - 41) Plazo extendido          2
(50 - 58) Muy largo plazo          2
(74 - 82) Larga temporada          2
(66 - 74) Extensivo                0
Name: count, dtype: int64

In [265]:

#Calculamos el número total de la población "n"

dfSturges['maximum_nights_avg_ntm'].info()
n = len(dfSturges['maximum_nights_avg_ntm'])
n
# n de CDMX: 26,536
# n de Seattle: 6,442

# Obtener el límite superior y el límite inferior de la columna objetivo
Max = dfSturges["maximum_nights_avg_ntm"].max()
Min = dfSturges["maximum_nights_avg_ntm"].min()
Limites = [Min, Max]
Limites

# Calculamos el rango R
R = Max - Min
R

# Regla de Sturges: Calcular el número de Intervalos de Clase "ni"
ni = 1 + 3.32*np.log10(n)
ni = int(ni)
ni

# Calculamos el ancho del intervalo "i"
i = R/ni
i = round(i,0)
i

# Categorización de variables
intervalos = np.linspace(Min, Max, ni)
intervalos = np.round(intervalos, 0).astype(int)
intervalos

categorias = [ 
    '(1 - 103) Poco tiempo',
    '(103 - 205) Corto plazo',
    '(205 - 308) Plazo moderado',
    '(308 - 410) Plazo extendido',
    '(410 - 512) Largo plazo',
    '(512 - 614) Prolongado',
    '(614 - 716) Extensivo',
    '(716 - 818) Amplio',
    '(818 - 921) Temporal',
    '(921 - 1023) Anual',
    '(1023 - 1125) Máximo'
]

dfSturges['maximumNightsCategory'] = pd.cut(x = dfSturges['maximum_nights_avg_ntm'], bins = intervalos, labels = categorias, include_lowest=True)
dfSturges['maximumNightsCategory'].value_counts()

<class 'pandas.core.series.Series'>
RangeIndex: 2862 entries, 0 to 2861
Series name: maximum_nights_avg_ntm
Non-Null Count  Dtype  
--------------  -----  
2862 non-null   float64
dtypes: float64(1)
memory usage: 22.5 KB


C:\Users\Angel González\AppData\Local\Temp\ipykernel_24188\1267331511.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfSturges['maximumNightsCategory'] = pd.cut(x = dfSturges['maximum_nights_avg_ntm'], bins = intervalos, labels = categorias, include_lowest=True)


maximumNightsCategory
(1023 - 1125) Máximo           1239
(308 - 410) Plazo extendido     858
(1 - 103) Poco tiempo           536
(103 - 205) Corto plazo         132
(716 - 818) Amplio               50
(205 - 308) Plazo moderado       23
(410 - 512) Largo plazo           8
(614 - 716) Extensivo             7
(512 - 614) Prolongado            4
(921 - 1023) Anual                3
(818 - 921) Temporal              2
Name: count, dtype: int64

In [266]:

#Calculamos el número total de la población "n"

dfSturges['availability_365'].info()
n = len(dfSturges['availability_365'])
n
# n de CDMX: 26,536
# n de Seattle: 6,442

# Obtener el límite superior y el límite inferior de la columna objetivo
Max = dfSturges["availability_365"].max()
Min = dfSturges["availability_365"].min()
Limites = [Min, Max]
Limites

# Calculamos el rango R
R = Max - Min
R

# Regla de Sturges: Calcular el número de Intervalos de Clase "ni"
ni = 1 + 3.32*np.log10(n)
ni = int(ni)
ni

# Calculamos el ancho del intervalo "i"
i = R/ni
i = round(i,0)
i

# Categorización de variables
intervalos = np.linspace(Min, Max, ni)
intervalos = np.round(intervalos, 0).astype(int)

intervalos = [0,  30,  61,  91, 122, 152, 182, 213, 243, 274, 304, 335, 365]

categorias = [ 
    '(0 - 30) Muy baja disponibilidad',
    '(30 - 61) Baja disponibilidad',
    '(61 - 91) Disponibilidad sencilla',
    '(91 - 122) Disponibilidad moderada',
    '(122 - 152) Disponibilidad regular',
    '(152 - 182) Disponibilidad alta',
    '(182 - 213) Muy alta disponibilidad',
    '(213 - 243) Extensa disponibilidad',
    '(243 - 274) Casi total disponibilidad',
    '(274 - 304) Disponibilidad casi completa',
    '(304 - 335) Total disponibilidad',
    '(335 - 365) Año Completo'
]

dfSturges['availabilityYearCat'] = pd.cut(x = dfSturges['availability_365'], bins = intervalos, labels = categorias, include_lowest=True)
dfSturges['availabilityYearCat'].value_counts()

<class 'pandas.core.series.Series'>
RangeIndex: 2862 entries, 0 to 2861
Series name: availability_365
Non-Null Count  Dtype
--------------  -----
2862 non-null   int64
dtypes: int64(1)
memory usage: 22.5 KB


C:\Users\Angel González\AppData\Local\Temp\ipykernel_24188\3695046190.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfSturges['availabilityYearCat'] = pd.cut(x = dfSturges['availability_365'], bins = intervalos, labels = categorias, include_lowest=True)


availabilityYearCat
(335 - 365) Año Completo                    451
(243 - 274) Casi total disponibilidad       369
(61 - 91) Disponibilidad sencilla           338
(152 - 182) Disponibilidad alta             246
(0 - 30) Muy baja disponibilidad            243
(304 - 335) Total disponibilidad            228
(30 - 61) Baja disponibilidad               197
(274 - 304) Disponibilidad casi completa    191
(213 - 243) Extensa disponibilidad          167
(122 - 152) Disponibilidad regular          155
(182 - 213) Muy alta disponibilidad         141
(91 - 122) Disponibilidad moderada          136
Name: count, dtype: int64

In [267]:

#Calculamos el número total de la población "n"

dfSturges['number_of_reviews'].info()
n = len(dfSturges['number_of_reviews'])
n
# n de CDMX: 26,536
# n de Seattle: 6,442

# Obtener el límite superior y el límite inferior de la columna objetivo
Max = dfSturges["number_of_reviews"].max()
Min = dfSturges["number_of_reviews"].min()
Limites = [Min, Max]
Limites

# Calculamos el rango R
R = Max - Min
R

# Regla de Sturges: Calcular el número de Intervalos de Clase "ni"
ni = 1 + 3.32*np.log10(n)
ni = int(ni)
ni

# Calculamos el ancho del intervalo "i"
i = R/ni
i = round(i,0)
i

# Categorización de variables
intervalos = np.linspace(Min, Max, ni)
intervalos = np.round(intervalos, 0).astype(int)
intervalos

categorias = [ 
    '(0 - 25) Muy pocas reseñas',
    '(25 - 50) Pocas reseñas',
    '(50 - 75) Reseñas limitadas',
    '(75 - 100) Reseñas moderadas',
    '(100 - 125) Buen número de reseñas',
    '(125 - 150) Reseñas sólidas',
    '(150 - 175) Muchas reseñas',
    '(175 - 200) Con mucho flujo',
    '(200 - 225) Muy comentado',
    '(225 - 250) Popular',
    '(250 - 275) Muy popular'
]

dfSturges['reviewsNumCat'] = pd.cut(x = dfSturges['number_of_reviews'], bins = intervalos, labels = categorias, include_lowest=True)
dfSturges['reviewsNumCat'].value_counts()


<class 'pandas.core.series.Series'>
RangeIndex: 2862 entries, 0 to 2861
Series name: number_of_reviews
Non-Null Count  Dtype
--------------  -----
2862 non-null   int64
dtypes: int64(1)
memory usage: 22.5 KB


C:\Users\Angel González\AppData\Local\Temp\ipykernel_24188\3239663982.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfSturges['reviewsNumCat'] = pd.cut(x = dfSturges['number_of_reviews'], bins = intervalos, labels = categorias, include_lowest=True)


reviewsNumCat
(0 - 25) Muy pocas reseñas            1763
(25 - 50) Pocas reseñas                445
(50 - 75) Reseñas limitadas            198
(75 - 100) Reseñas moderadas           166
(100 - 125) Buen número de reseñas      98
(125 - 150) Reseñas sólidas             63
(150 - 175) Muchas reseñas              50
(200 - 225) Muy comentado               26
(175 - 200) Con mucho flujo             23
(225 - 250) Popular                     15
(250 - 275) Muy popular                 15
Name: count, dtype: int64

In [268]:

#Calculamos el número total de la población "n"

dfSturges['host_years'].info()
n = len(dfSturges['host_years'])
n
# n de CDMX: 26,536
# n de Seattle: 6,442

# Obtener el límite superior y el límite inferior de la columna objetivo
Max = dfSturges["host_years"].max()
Min = dfSturges["host_years"].min()
Limites = [Min, Max]
Limites

# Calculamos el rango R
R = Max - Min
R

# Regla de Sturges: Calcular el número de Intervalos de Clase "ni"
ni = 1 + 3.32*np.log10(n)
ni = int(ni)
ni

# Calculamos el ancho del intervalo "i"
i = R/ni
i = round(i,0)
i

# Categorización de variables
intervalos = np.linspace(Min, Max, ni)
intervalos = np.round(intervalos, 0).astype(int)
intervalos = [ 0, 4, 8, 12, 16]

categorias = ['Principiante (0-3 años)','Intermedio (4-7 años)','Profesional (8-11 años)','Experimentado (12-16 años)']

# Finalmente creamos las categorías en la columna numérica
dfSturges['hostYearsCat'] = pd.cut(x = dfSturges['host_years'], bins = intervalos, labels = categorias, include_lowest=True)
dfSturges['hostYearsCat'].value_counts()


<class 'pandas.core.series.Series'>
RangeIndex: 2862 entries, 0 to 2861
Series name: host_years
Non-Null Count  Dtype
--------------  -----
2862 non-null   int64
dtypes: int64(1)
memory usage: 22.5 KB


C:\Users\Angel González\AppData\Local\Temp\ipykernel_24188\2159768127.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfSturges['hostYearsCat'] = pd.cut(x = dfSturges['host_years'], bins = intervalos, labels = categorias, include_lowest=True)


hostYearsCat
Intermedio (4-7 años)         1173
Principiante (0-3 años)        968
Profesional (8-11 años)        696
Experimentado (12-16 años)      25
Name: count, dtype: int64

In [269]:

#Calculamos el número total de la población "n"

dfSturges['priceMex'].info()
n = len(dfSturges['priceMex'])
n
# n de Seattle: 6,442

# Obtener el límite superior y el límite inferior de la columna objetivo
Max = dfSturges["priceMex"].max()
Min = dfSturges["priceMex"].min()
Limites = [Min, Max]
Limites

# Calculamos el rango R
R = Max - Min
R

# Regla de Sturges: Calcular el número de Intervalos de Clase "ni"
ni = 1 + 3.32*np.log10(n)
ni = int(ni)
ni

# Calculamos el ancho del intervalo "i"
i = R/ni
i = round(i,0)
i

# Categorización de variables
intervalos = np.linspace(Min, Max, ni)
intervalos = np.round(intervalos, 0).astype(int)
intervalos

categorias = [ 
    '(320 - 1,184) Muy bajo precio',
    '(1,184 - 2,047) Bajo precio',
    '(2,047 - 2,911) Precio moderado',
    '(2,911 - 3,774) Precio razonable',
    '(3,774 - 4,638) Precio alto',
    '(4,638 - 5,501) Precio elevado',
    '(5,501 - 6,365) Alto precio',
    '(6,365 - 7,228) Muy alto precio',
    '(7,228 - 8,092) Precio premium',
    '(8,092 - 8,955) Precio de lujo',
    '(8,955 - 9,819) Precio máximo'
]

# Finalmente creamos las categorías en la columna numérica
dfSturges['priceMexCat'] = pd.cut(x = dfSturges['priceMex'], bins = intervalos, labels = categorias, include_lowest=True)
dfSturges['priceMexCat'].value_counts()


<class 'pandas.core.series.Series'>
RangeIndex: 2862 entries, 0 to 2861
Series name: priceMex
Non-Null Count  Dtype  
--------------  -----  
2862 non-null   float64
dtypes: float64(1)
memory usage: 22.5 KB


C:\Users\Angel González\AppData\Local\Temp\ipykernel_24188\731225933.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfSturges['priceMexCat'] = pd.cut(x = dfSturges['priceMex'], bins = intervalos, labels = categorias, include_lowest=True)


priceMexCat
(1,184 - 2,047) Bajo precio         1044
(320 - 1,184) Muy bajo precio        859
(2,047 - 2,911) Precio moderado      389
(2,911 - 3,774) Precio razonable     264
(3,774 - 4,638) Precio alto          131
(4,638 - 5,501) Precio elevado        70
(5,501 - 6,365) Alto precio           43
(6,365 - 7,228) Muy alto precio       26
(8,092 - 8,955) Precio de lujo        18
(7,228 - 8,092) Precio premium        13
(8,955 - 9,819) Precio máximo          5
Name: count, dtype: int64

In [270]:

#Calculamos el número total de la población "n"

dfSturges['host_response_rate'].info()
n = len(dfSturges['host_response_rate'])
n

# Obtener el límite superior y el límite inferior de la columna objetivo
Max = dfSturges["host_response_rate"].max()
Min = dfSturges["host_response_rate"].min()
Limites = [Min, Max]
Limites

# Calculamos el rango R
R = Max - Min
R

# Regla de Sturges: Calcular el número de Intervalos de Clase "ni"
ni = 1 + 3.32*np.log10(n)
ni = int(ni)
ni

# Calculamos el ancho del intervalo "i"
i = R/ni
i = round(i,0)
i

# Categorización de variables
intervalos = np.linspace(Min, Max, ni)
intervalos = np.round(intervalos, 0).astype(int)
intervalos

categorias = [ 
    '(0 - 9%) Muy baja tasa de respuesta',
    '(9 - 18%) Baja tasa de respuesta',
    '(18 - 27%) Tasa de respuesta limitada',
    '(27 - 36%) Tasa de respuesta moderada',
    '(36 - 45%) Tasa de respuesta adecuada',
    '(45 - 55%) Tasa de respuesta buena',
    '(55 - 64%) Alta tasa de respuesta',
    '(64 - 73%) Muy buena tasa de respuesta',
    '(73 - 82%) Excelente tasa de respuesta',
    '(82 - 91%) Tasa de respuesta sobresaliente',
    '(91 - 100%) Tasa de respuesta perfecta'
]

# Finalmente creamos las categorías en la columna numérica
dfSturges['hostResponseCat'] = pd.cut(x = dfSturges['host_response_rate'], bins = intervalos, labels = categorias, include_lowest=True)
dfSturges['hostResponseCat'].value_counts()

<class 'pandas.core.series.Series'>
RangeIndex: 2862 entries, 0 to 2861
Series name: host_response_rate
Non-Null Count  Dtype  
--------------  -----  
2862 non-null   float64
dtypes: float64(1)
memory usage: 22.5 KB


C:\Users\Angel González\AppData\Local\Temp\ipykernel_24188\1392730686.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfSturges['hostResponseCat'] = pd.cut(x = dfSturges['host_response_rate'], bins = intervalos, labels = categorias, include_lowest=True)


hostResponseCat
(91 - 100%) Tasa de respuesta perfecta        2503
(82 - 91%) Tasa de respuesta sobresaliente     176
(0 - 9%) Muy baja tasa de respuesta             55
(73 - 82%) Excelente tasa de respuesta          44
(64 - 73%) Muy buena tasa de respuesta          31
(45 - 55%) Tasa de respuesta buena              20
(9 - 18%) Baja tasa de respuesta                19
(18 - 27%) Tasa de respuesta limitada            7
(27 - 36%) Tasa de respuesta moderada            4
(55 - 64%) Alta tasa de respuesta                2
(36 - 45%) Tasa de respuesta adecuada            1
Name: count, dtype: int64

In [271]:

#Calculamos el número total de la población "n"

dfSturges['host_acceptance_rate'].info()
n = len(dfSturges['host_acceptance_rate'])
n
# n de Seattle: 6,442

# Obtener el límite superior y el límite inferior de la columna objetivo
Max = dfSturges["host_acceptance_rate"].max()
Min = dfSturges["host_acceptance_rate"].min()
Limites = [Min, Max]
Limites

# Limites de CDMX: [0, 100]
# Límites de Seattle: [0, 100]

# Calculamos el rango R
R = Max - Min
R

# Regla de Sturges: Calcular el número de Intervalos de Clase "ni"
ni = 1 + 3.32*np.log10(n)
ni = int(ni)
ni

# Dado que los extremos de los límites más altos son los de Ottawa, se optará por manejar esta escala

# Calculamos el ancho del intervalo "i"
i = R/ni
i = round(i,0)
i

# Categorización de variables
intervalos = np.linspace(Min, Max, ni)
intervalos = np.round(intervalos, 0)
intervalos



# Creamos las categorías
categorias = [
    '(0 - 9%) Muy Malo',
    '(10 - 18%) Malo',
    '(19 - 27%) Muy Bajo',
    '(28 - 36%) Bajo',
    '(37 - 45%) Medio',
    '(46 - 55%) Mejorable',
    '(56 - 64%) Aceptable',
    '(65 - 73%) Bueno',
    '(74 - 82%) Muy bueno',
    '(83 - 91%) Alto',
    '(92 - 100%) Excelente'
]

# Finalmente creamos las categorías en la columna numérica
dfSturges['hostAcceptanceCat'] = pd.cut(x = dfSturges['host_acceptance_rate'], bins = intervalos, labels = categorias, include_lowest=True)
dfSturges['hostAcceptanceCat'].value_counts()

<class 'pandas.core.series.Series'>
RangeIndex: 2862 entries, 0 to 2861
Series name: host_acceptance_rate
Non-Null Count  Dtype  
--------------  -----  
2862 non-null   float64
dtypes: float64(1)
memory usage: 22.5 KB


hostAcceptanceCat
(92 - 100%) Excelente    1701
(83 - 91%) Alto           655
(74 - 82%) Muy bueno      155
(0 - 9%) Muy Malo         102
(65 - 73%) Bueno           86
(46 - 55%) Mejorable       57
(56 - 64%) Aceptable       46
(28 - 36%) Bajo            25
(19 - 27%) Muy Bajo        17
(37 - 45%) Medio           14
(10 - 18%) Malo             4
Name: count, dtype: int64

In [272]:
#Calculamos el número total de la población "n"

dfSturges['host_verifications_num'].info()
n = len(dfSturges['host_verifications_num'])
n
# n de Seattle: 6,442

# Obtener el límite superior y el límite inferior de la columna objetivo
Max = dfSturges["host_verifications_num"].max()
Min = dfSturges["host_verifications_num"].min()
Limites = [Min, Max]
Limites

# Calculamos el rango R
R = Max - Min
R

# Regla de Sturges: Calcular el número de Intervalos de Clase "ni"
ni = 1 + 3.32*np.log10(n)
ni = int(ni)
ni

# Dado que los extremos de los límites más altos son los de Ottawa, se optará por manejar esta escala

# Calculamos el ancho del intervalo "i"
i = R/ni
i = round(i,0)
i

# Categorización de variables
intervalos = np.linspace(Min, Max, ni)
intervalos = np.round(intervalos, 0)

intervalos = [0,1,2,3]

categorias = [ 
    '(0 - 1) Verificación básica',
    '(2) Verificación moderada',
    '(3) Verificación completa'
]

# Finalmente creamos las categorías en la columna numérica
dfSturges['hostVerifCat'] = pd.cut(x = dfSturges['host_verifications_num'], bins = intervalos, labels = categorias, include_lowest=True)
dfSturges['hostVerifCat'].value_counts()

<class 'pandas.core.series.Series'>
RangeIndex: 2862 entries, 0 to 2861
Series name: host_verifications_num
Non-Null Count  Dtype
--------------  -----
2862 non-null   int64
dtypes: int64(1)
memory usage: 22.5 KB


hostVerifCat
(2) Verificación moderada      2256
(3) Verificación completa       352
(0 - 1) Verificación básica     254
Name: count, dtype: int64

In [273]:
# La columna de host_response_time ya fue categorizada previamente
# df[["host_response_time","host_response_time_num"]].value_counts()

In [274]:
# La columna de bathroom_text ya fue categorizada previamente
# df[["bathroom_text_clean","bathroom_text_num"]].value_counts()

In [275]:
# La columna de neighbourhood_cleansed ya fue categorizada previamente
# df[["neighbourhood_Category","neighbourhood_Category_num"]].value_counts()

In [276]:
#Calculamos el número total de la población "n"

dfSturges['beds'].info()
n = len(dfSturges['beds'])
n
# n de Seattle: 6,442

# Obtener el límite superior y el límite inferior de la columna objetivo
Max = dfSturges["beds"].max()
Min = dfSturges["beds"].min()
Limites = [Min, Max]
Limites

# Calculamos el rango R
R = Max - Min
R

# Regla de Sturges: Calcular el número de Intervalos de Clase "ni"
ni = 1 + 3.32*np.log10(n)
ni = int(ni)
ni

# Dado que los extremos de los límites más altos son los de Ottawa, se optará por manejar esta escala

# Calculamos el ancho del intervalo "i"
i = R/ni
i = round(i,0)
i

intervalos = [0,1,3,5,6]

categorias = [ 
    '(0 - 1) Una cama',
    '(2) Pocas camas',
    '(3 - 5) Camas múltiples',
    '(5 - 6) Muchas camas'
]

# Finalmente creamos las categorías en la columna numérica
dfSturges['bedsCategory'] = pd.cut(x = dfSturges['beds'], bins = intervalos, labels = categorias, include_lowest=True)
dfSturges['bedsCategory'].value_counts()

<class 'pandas.core.series.Series'>
RangeIndex: 2862 entries, 0 to 2861
Series name: beds
Non-Null Count  Dtype
--------------  -----
2862 non-null   int64
dtypes: int64(1)
memory usage: 22.5 KB


bedsCategory
(0 - 1) Una cama           1748
(2) Pocas camas             818
(3 - 5) Camas múltiples     250
(5 - 6) Muchas camas         46
Name: count, dtype: int64

In [277]:
# Usando el mínimo de noches promedio
#Calculamos el número total de la población "n"

dfSturges['reviews_per_month'].info()
n = len(dfSturges['reviews_per_month'])
n
# n de Seattle: 6,442

# Obtener el límite superior y el límite inferior de la columna objetivo
Max = dfSturges["reviews_per_month"].max()
Min = dfSturges["reviews_per_month"].min()
Limites = [Min, Max]
Limites

# Calculamos el rango R
R = Max - Min
R

# Regla de Sturges: Calcular el número de Intervalos de Clase "ni"
ni = 1 + 3.32*np.log10(n)
ni = int(ni)
ni

# Dado que los extremos de los límites más altos son los de Ottawa, se optará por manejar esta escala

# Calculamos el ancho del intervalo "i"
i = R/ni
i = round(i,0)
i

# Categorización de variables
intervalos = np.linspace(Min, Max, ni)
intervalos = np.round(intervalos, 0)

intervalos = [0,1,3,5,7,10]

categorias = [ 
    '(0) Sin reseñas',
    '(1) Reseña mensual mínima',
    '(2 - 5) Reseñas mensuales moderadas',
    '(5 - 7) Buen número de reseñas mensuales',
    '(7 - 10) Alta frecuencia de reseñas'
]

# Finalmente creamos las categorías en la columna numérica
dfSturges['reviewsMonthCat'] = pd.cut(x = dfSturges['reviews_per_month'], bins = intervalos, labels = categorias, include_lowest=True)
dfSturges['reviewsMonthCat'].value_counts()

<class 'pandas.core.series.Series'>
RangeIndex: 2862 entries, 0 to 2861
Series name: reviews_per_month
Non-Null Count  Dtype  
--------------  -----  
2862 non-null   float64
dtypes: float64(1)
memory usage: 22.5 KB


reviewsMonthCat
(1) Reseña mensual mínima                   1215
(0) Sin reseñas                             1118
(2 - 5) Reseñas mensuales moderadas          329
(5 - 7) Buen número de reseñas mensuales     138
(7 - 10) Alta frecuencia de reseñas           62
Name: count, dtype: int64

## Escalas Likert

Para las escalas Likert, dado que ya son variables que significan categorías estás se procesarán manualmente

In [278]:
# Usando el mínimo de noches promedio
#Calculamos el número total de la población "n"

dfSturges['review_scores_value'].info()
n = len(dfSturges['review_scores_value'])
n
# n de Seattle: 6,442

# Obtener el límite superior y el límite inferior de la columna objetivo
Max = dfSturges["review_scores_value"].max()
Min = dfSturges["review_scores_value"].min()
Limites = [Min, Max]
Limites

intervalos = [1,2,3,4,5]

categorias = [ 
    '(1 - 2) Insatisfecho',
    '(3) Neutral',
    '(4) Satisfecho',
    '(5) Muy satisfecho'
]

# Finalmente creamos las categorías en la columna numérica
dfSturges['reviewScoresCat'] = pd.cut(x = dfSturges['review_scores_value'], bins = intervalos, labels = categorias, include_lowest=True)
dfSturges['reviewScoresCat'].value_counts()

<class 'pandas.core.series.Series'>
RangeIndex: 2862 entries, 0 to 2861
Series name: review_scores_value
Non-Null Count  Dtype
--------------  -----
2862 non-null   int64
dtypes: int64(1)
memory usage: 22.5 KB


reviewScoresCat
(5) Muy satisfecho      2508
(4) Satisfecho           312
(3) Neutral               26
(1 - 2) Insatisfecho      16
Name: count, dtype: int64

In [279]:
# Usando el mínimo de noches promedio
#Calculamos el número total de la población "n"

dfSturges['review_scores_location'].info()
n = len(dfSturges['review_scores_location'])
n
# n de Seattle: 6,442

# Obtener el límite superior y el límite inferior de la columna objetivo
Max = dfSturges["review_scores_location"].max()
Min = dfSturges["review_scores_location"].min()
Limites = [Min, Max]
Limites

intervalos = [1,2,3,4,5]

categorias = [ 
    '(1 - 2) Insatisfecho',
    '(3) Neutral',
    '(4) Satisfecho',
    '(5) Muy satisfecho'
]

# Finalmente creamos las categorías en la columna numérica
dfSturges['reviewLocationCat'] = pd.cut(x = dfSturges['review_scores_location'], bins = intervalos, labels = categorias, include_lowest=True)
dfSturges['reviewLocationCat'].value_counts()

<class 'pandas.core.series.Series'>
RangeIndex: 2862 entries, 0 to 2861
Series name: review_scores_location
Non-Null Count  Dtype
--------------  -----
2862 non-null   int64
dtypes: int64(1)
memory usage: 22.5 KB


reviewLocationCat
(5) Muy satisfecho      2643
(4) Satisfecho           200
(3) Neutral               13
(1 - 2) Insatisfecho       6
Name: count, dtype: int64

In [280]:
# Usando el mínimo de noches promedio
#Calculamos el número total de la población "n"

dfSturges['review_scores_rating'].info()
n = len(dfSturges['review_scores_rating'])
n
# n de Seattle: 6,442

# Obtener el límite superior y el límite inferior de la columna objetivo
Max = dfSturges["review_scores_rating"].max()
Min = dfSturges["review_scores_rating"].min()
Limites = [Min, Max]
Limites

intervalos = [1,2,3,4,5]

categorias = [ 
    '(1 - 2) Insatisfecho',
    '(3) Neutral',
    '(4) Satisfecho',
    '(5) Muy satisfecho'
]

# Finalmente creamos las categorías en la columna numérica
dfSturges['reviewRatingCat'] = pd.cut(x = dfSturges['review_scores_rating'], bins = intervalos, labels = categorias, include_lowest=True)
dfSturges['reviewRatingCat'].value_counts()

<class 'pandas.core.series.Series'>
RangeIndex: 2862 entries, 0 to 2861
Series name: review_scores_rating
Non-Null Count  Dtype
--------------  -----
2862 non-null   int64
dtypes: int64(1)
memory usage: 22.5 KB


reviewRatingCat
(5) Muy satisfecho      2605
(4) Satisfecho           225
(3) Neutral               22
(1 - 2) Insatisfecho      10
Name: count, dtype: int64

In [281]:
# Usando el mínimo de noches promedio
#Calculamos el número total de la población "n"

dfSturges['review_scores_cleanliness'].info()
n = len(dfSturges['review_scores_cleanliness'])
n
# n de Seattle: 6,442

# Obtener el límite superior y el límite inferior de la columna objetivo
Max = dfSturges["review_scores_cleanliness"].max()
Min = dfSturges["review_scores_cleanliness"].min()
Limites = [Min, Max]
Limites

intervalos = [1,2,3,4,5]

categorias = [ 
    '(1 - 2) Insatisfecho',
    '(3) Neutral',
    '(4) Satisfecho',
    '(5) Muy satisfecho'
]
# Finalmente creamos las categorías en la columna numérica
dfSturges['reviewCleanCat'] = pd.cut(x = dfSturges['review_scores_cleanliness'], bins = intervalos, labels = categorias, include_lowest=True)
dfSturges['reviewCleanCat'].value_counts()

<class 'pandas.core.series.Series'>
RangeIndex: 2862 entries, 0 to 2861
Series name: review_scores_cleanliness
Non-Null Count  Dtype
--------------  -----
2862 non-null   int64
dtypes: int64(1)
memory usage: 22.5 KB


reviewCleanCat
(5) Muy satisfecho      2560
(4) Satisfecho           270
(3) Neutral               17
(1 - 2) Insatisfecho      15
Name: count, dtype: int64

In [282]:
# Usando el mínimo de noches promedio
#Calculamos el número total de la población "n"

dfSturges['review_scores_communication'].info()
n = len(dfSturges['review_scores_communication'])
n
# n de Seattle: 6,442

# Obtener el límite superior y el límite inferior de la columna objetivo
Max = dfSturges["review_scores_communication"].max()
Min = dfSturges["review_scores_communication"].min()
Limites = [Min, Max]
Limites

intervalos = [1,2,3,4,5]

categorias = [ 
    '(1 - 2) Insatisfecho',
    '(3) Neutral',
    '(4) Satisfecho',
    '(5) Muy satisfecho'
]

# Finalmente creamos las categorías en la columna numérica
dfSturges['reviewCommunicationCat'] = pd.cut(x = dfSturges['review_scores_communication'], bins = intervalos, labels = categorias, include_lowest=True)
dfSturges['reviewCommunicationCat'].value_counts()

<class 'pandas.core.series.Series'>
RangeIndex: 2862 entries, 0 to 2861
Series name: review_scores_communication
Non-Null Count  Dtype
--------------  -----
2862 non-null   int64
dtypes: int64(1)
memory usage: 22.5 KB


reviewCommunicationCat
(5) Muy satisfecho      2735
(4) Satisfecho           102
(3) Neutral               14
(1 - 2) Insatisfecho      11
Name: count, dtype: int64

In [283]:
# Comprobamos que no tenemos nulos
dfSturges.isnull().sum().sum()

0

In [284]:
# Unimos las columnas de Sturges y las del dataframe original
dfUnido = pd.concat([df, dfSturges], axis=1)

In [285]:
# Comprobamos que no tengamos nulos 
dfUnido.isnull().sum().sum()

0

In [286]:
# Convertir Dataframe a CSV
dfUnido.to_excel("OttawaSturges.xlsx")